# Sequential Recommender Quick Start
Sequential recommender는 유저 행동의 시퀀스를 문맥처럼 활용하고 유저가 짧은 기간 안에 상호작용할 아이템을 예측하는 것을 목표로 한다.   
일반성을 잃지 않고, 이 노트는 `SLi-Rec model`을 예제로 본다. `SLi-Rec model`은 정확한 추천을 위해 장/단기 유저 선호를 모두 사용하는 딥러닝 기반 모델이다. 이 모델의 장점은 다음과 같다.
- 장기 모델링을 위해 attentive 'Asymmetric-SVD' 패러다임을 적용한다.
- LSTM에서 게이트 로직을 수정함으로써 시간 불규칙성과 의미 불규칙성을 고려한다.
- 장기 구성 요소와 단기 구성 요소를 동적으로 결합하기 위해 어텐션 메커니즘을 사용한다.

In [1]:
import sys
import os
import logging
import papermill as pm
import scrapbook as sb
from tempfile import TemporaryDirectory
import numpy as np
import tensorflow.compat.v1 as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.models.deeprec.deeprec_utils import prepare_hparams
from recommenders.datasets.amazon_reviews import download_and_extract, data_preprocessing
from recommenders.datasets.download_utils import maybe_download
from recommenders.models.deeprec.models.sequential.sli_rec import SLI_RECModel as SeqModel
####  to use the other model, use one of the following lines:
# from recommenders.models.deeprec.models.sequential.asvd import A2SVDModel as SeqModel
# from recommenders.models.deeprec.models.sequential.caser import CaserModel as SeqModel
# from recommenders.models.deeprec.models.sequential.gru4rec import GRU4RecModel as SeqModel
# from recommenders.models.deeprec.models.sequential.sum import SUMModel as SeqModel
# from recommenders.models.deeprec.models.sequential.nextitnet import NextItNetModel

from recommenders.models.deeprec.io.sequential_iterator import SequentialIterator
# from recommenders.models.deeprec.io.nextitnet_iterator import NextItNetIterator

In [5]:
##  ATTENTION: change to the corresponding config file, e.g., caser.yaml for CaserModel, sum.yaml for SUMModel
yaml_file = '../../recommenders/models/deeprec/config/sli_rec.yaml'  

## Input data format
Input data는 8개의 칼럼을 갖고 있다.

## Amazon dataset

In [2]:
data_path = os.path.join('..','..','tests','resources','deeprec','slirec')

# for test
train_file = os.path.join(data_path, r'train_data')
valid_file = os.path.join(data_path, r'valid_data')
test_file = os.path.join(data_path, r'test_data')
user_vocab = os.path.join(data_path, r'user_vocab.pkl')
item_vocab = os.path.join(data_path, r'item_vocab.pkl')
cate_vocab = os.path.join(data_path, r'category_vocab.pkl')
output_file = os.path.join(data_path, r'output.txt')

reviews_name = 'reviews_Movies_and_TV_5.json'
meta_name = 'meta_Movies_and_TV.json'
reviews_file = os.path.join(data_path, reviews_name)
meta_file = os.path.join(data_path, meta_name)
train_num_ngs = 4 # number of negative instances with a positive instance for training
valid_num_ngs = 4 # number of negative instances with a positive instance for validation
test_num_ngs = 9 # number of negative instances with a positive instance for testing
sample_rate = 0.01 # sample a small item set for training and testing here for fast example

input_files = [reviews_file, meta_file, train_file, valid_file, test_file, user_vocab, item_vocab, cate_vocab]

if not os.path.exists(train_file):
    download_and_extract(reviews_name, reviews_file)
    download_and_extract(meta_name, meta_file)
    data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)
    #### uncomment this for the NextItNet model, because it does not need to unfold the user history
    # data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs, is_history_expanding=False)


In [6]:
hparams = prepare_hparams(yaml_file, embed_l2=0., layer_l2=0.,
                         learning_rate=0.001, epochs=10, batch_size=400,
                         show_step=20, MODEL_DIR=os.path.join(data_path,'model/'),
                         SUMMARIES_DIR=os.path.join(data_path, 'summary/'),
                         user_vocab=user_vocab, item_vocab=item_vocab,
                         cate_vocab=cate_vocab, need_sample=True,
                         train_num_ngs=train_num_ngs)


FileNotFoundError: [Errno 2] No such file or directory: '../../recommenders/models/deeprec/config/sli_rec.yaml'

In [4]:
data_path

'../../tests/resources/deeprec/slirec'